In [2]:
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Set random seed for reproducibility
np.random.seed(42)

# Function to simulate data based on a single effect size for all parameters
def simulate_data(n, effect_size):
    # Random data for IV, mediator, moderator
    IV = np.random.normal(size=n)
    M = effect_size * IV + np.random.normal(size=n)
    MOD = np.random.normal(size=n)
    
    # Interaction effects
    IV_MOD = IV * MOD
    IV_M = IV * M
    
    # Outcome (Y) with direct and indirect effects + noise
    Y = effect_size * IV + effect_size * M + effect_size * MOD + effect_size * IV_MOD + effect_size * IV_M + np.random.normal(size=n)
    
    # Create a dataset
    data = {
        'IV': IV,
        'M': M,
        'MOD': MOD,
        'IV_MOD': IV_MOD,
        'IV_M': IV_M,
        'Y': Y
    }
    
    return data

# Function to run a single regression model
def run_model(data):
    # Define the formula for the moderated mediation model
    formula = 'Y ~ IV + M + MOD + IV_MOD + IV_M'
    
    # Fit the model using OLS regression
    model = ols(formula, data=data).fit()
    
    return model

# Simulate multiple datasets to estimate power
def estimate_power(n_simulations, sample_size, effect_size):
    significant_count = 0
    
    for _ in range(n_simulations):
        data = simulate_data(sample_size, effect_size)
        model = run_model(data)
        
        # Check if interaction terms are significant
        if model.pvalues['IV_MOD'] < 0.05 and model.pvalues['IV_M'] < 0.05:
            significant_count += 1
    
    # Calculate the proportion of simulations where the effect was significant
    power = significant_count / n_simulations
    return power

# Function to find the sample size that achieves the desired power level
def find_sample_size(desired_power, n_simulations, max_sample_size, effect_size, power_step=0.01):
    sample_size = 50  # Start with a small sample size
    while sample_size <= max_sample_size:
        power = estimate_power(n_simulations, sample_size, effect_size)
        print(f"Sample size: {sample_size}, Power: {power:.2f}")
        
        if power >= desired_power:
            return sample_size, power
        
        # Increase sample size incrementally
        sample_size += 10
    
    return None, power

# Input parameters
desired_power = 0.80  # Target power level
n_simulations = 1000  # Number of simulations to run for each sample size
max_sample_size = 500  # Maximum sample size to consider

# Input your desired effect size for all parameters with error handling
while True:
    try:
        effect_size = float(input("Enter the effect size (for all variables): "))
        break  # If the input is valid, break out of the loop
    except ValueError:
        print("Invalid input. Please enter a valid number for the effect size.")

# Run the sample size search
optimal_sample_size, achieved_power = find_sample_size(desired_power, n_simulations, max_sample_size, effect_size)

if optimal_sample_size:
    print(f"Optimal sample size: {optimal_sample_size} with achieved power: {achieved_power:.2f}")
else:
    print(f"Could not achieve the desired power level within the max sample size of {max_sample_size}.")


Sample size: 50, Power: 0.24
Sample size: 60, Power: 0.31
Sample size: 70, Power: 0.40
Sample size: 80, Power: 0.51
Sample size: 90, Power: 0.58
Sample size: 100, Power: 0.67
Sample size: 110, Power: 0.71
Sample size: 120, Power: 0.77
Sample size: 130, Power: 0.83
Optimal sample size: 130 with achieved power: 0.83
